<a href="https://colab.research.google.com/github/btlgs2000/Elettric80/blob/main/model_selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# pip install optuna

In [ ]:
 # fashion mnist

# rete migliore come accuracy
# dropout rate (0, 0.5)
# numero di strati nascosti (1, 5)
# numero di unità per ogni strato nascosto (10, 200)
# relu e sigmoide
# learning rate 10-3 e 10-2 #Adam

from tensorflow.keras import Model, Sequential
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, Dropout, Flatten

import numpy as np

import optuna

In [ ]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [ ]:
x_train = (x_train / 255).astype(np.float32)
x_test = (x_test / 255).astype(np.float32)

In [ ]:
def get_model(droput_rate:float, num_hidden:int, num_units:int, activation:str) -> Model:
    ''' Restituisce un modello '''
    model = Sequential([Flatten()])

    # strati nascosti
    for _ in range(num_hidden):
        model.add(Dense(num_units, activation))

    # ultimo strato
    model.add(Dense(10, 'softmax'))

    return model

In [ ]:
class Objective:
    def __init__(self, x_train, y_train):
        self.x_train = x_train
        self.y_train = y_train

    def __call__(self, trial):
        # scelta iperparametri
        # trial.suggest....
        num_hidden = trial.suggest_int('num_hidden', 1, 5)
        num_units = trial.suggest_int('num_units', 10, 200)
        dropout_rate = trial.suggest_uniform('dropout_rate', 0., 0.5)
        learning_rate = trial.suggest_loguniform('learning_rate', 1e-3, 1e-2)
        activation = trial.suggest_categorical('activation', ['relu', 'sigmoid'])
        
        # get_mode
        model = get_model(dropout_rate, num_hidden, num_units, activation)
        model.compile(optimizer=Adam(learning_rate), loss='sparse_categorical_crossentropy', metrics='accuracy')
        
        # addestramento # 10 epoche con early stopping su accuratezza sul validation, patience=2
        model.fit(x=self.x_train, y=self.y_train, epochs=10, callbacks=[EarlyStopping(monitor='val_accuracy', patience=2)], validation_split=0.2)

        return max(model.history.history['val_accuracy'])
        # ritorna l'accuracy massima

In [ ]:
study = optuna.create_study(storage='sqlite:///model_selection.db', study_name='model_selection', direction='maximize')

[I 2021-05-21 10:25:28,278] A new study created in RDB with name: model_selection


In [ ]:
study.optimize(Objective(x_train, y_train), n_trials=200)

Epoch 1/10
1500/1500 [==============================] - 7s 4ms/step - loss: 0.6770 - accuracy: 0.7533 - val_loss: 0.4481 - val_accuracy: 0.8378
Epoch 2/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.4004 - accuracy: 0.8539 - val_loss: 0.3818 - val_accuracy: 0.8592
Epoch 3/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3538 - accuracy: 0.8700 - val_loss: 0.4426 - val_accuracy: 0.8482
Epoch 4/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3305 - accuracy: 0.8774 - val_loss: 0.3807 - val_accuracy: 0.8683
Epoch 5/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3110 - accuracy: 0.8860 - val_loss: 0.3398 - val_accuracy: 0.8775
Epoch 6/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.2978 - accuracy: 0.8871 - val_loss: 0.3469 - val_accuracy: 0.8736
Epoch 7/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.2864 - accuracy: 0.8939 - val_loss: 0.3474 - val_accuracy:

[I 2021-05-21 10:26:29,733] Trial 0 finished with value: 0.8888333439826965 and parameters: {'num_hidden': 4, 'num_units': 145, 'dropout_rate': 0.2548654541841737, 'learning_rate': 0.0012313862720931623, 'activation': 'relu'}. Best is trial 0 with value: 0.8888333439826965.


Epoch 1/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.6457 - accuracy: 0.7647 - val_loss: 0.4613 - val_accuracy: 0.8339
Epoch 2/10
1500/1500 [==============================] - 5s 3ms/step - loss: 0.4098 - accuracy: 0.8519 - val_loss: 0.3956 - val_accuracy: 0.8601
Epoch 3/10
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3711 - accuracy: 0.8662 - val_loss: 0.4031 - val_accuracy: 0.8547
Epoch 4/10
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3468 - accuracy: 0.8723 - val_loss: 0.3804 - val_accuracy: 0.8653
Epoch 5/10
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3310 - accuracy: 0.8763 - val_loss: 0.3715 - val_accuracy: 0.8687
Epoch 6/10
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3296 - accuracy: 0.8778 - val_loss: 0.3724 - val_accuracy: 0.8686
Epoch 7/10
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3230 - accuracy: 0.8799 - val_loss: 0.3529 - val_accuracy:

[I 2021-05-21 10:27:20,178] Trial 1 finished with value: 0.8829166889190674 and parameters: {'num_hidden': 2, 'num_units': 151, 'dropout_rate': 0.05184001043287578, 'learning_rate': 0.004683496069592317, 'activation': 'relu'}. Best is trial 0 with value: 0.8888333439826965.


Epoch 1/10
1500/1500 [==============================] - 7s 4ms/step - loss: 0.7529 - accuracy: 0.7277 - val_loss: 0.4826 - val_accuracy: 0.8274
Epoch 2/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.4720 - accuracy: 0.8315 - val_loss: 0.5026 - val_accuracy: 0.8268
Epoch 3/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.4642 - accuracy: 0.8359 - val_loss: 0.4465 - val_accuracy: 0.8438
Epoch 4/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.4265 - accuracy: 0.8483 - val_loss: 0.4590 - val_accuracy: 0.8501
Epoch 5/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.4133 - accuracy: 0.8534 - val_loss: 0.4283 - val_accuracy: 0.8493
Epoch 6/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3893 - accuracy: 0.8630 - val_loss: 0.4171 - val_accuracy: 0.8521
Epoch 7/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3948 - accuracy: 0.8602 - val_loss: 0.4147 - val_accuracy:

...

Epoch 1/10
1500/1500 [==============================] - 5s 3ms/step - loss: 0.7220 - accuracy: 0.7586 - val_loss: 0.4295 - val_accuracy: 0.8448
Epoch 2/10
1500/1500 [==============================] - 4s 3ms/step - loss: 0.4024 - accuracy: 0.8541 - val_loss: 0.4080 - val_accuracy: 0.8518
Epoch 3/10
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3631 - accuracy: 0.8679 - val_loss: 0.3528 - val_accuracy: 0.8734
Epoch 4/10
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3357 - accuracy: 0.8781 - val_loss: 0.3389 - val_accuracy: 0.8748
Epoch 5/10
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3135 - accuracy: 0.8848 - val_loss: 0.3387 - val_accuracy: 0.8774
Epoch 6/10
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2966 - accuracy: 0.8923 - val_loss: 0.3116 - val_accuracy: 0.8830
Epoch 7/10
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2773 - accuracy: 0.8977 - val_loss: 0.3175 - val_accuracy:

[I 2021-05-21 10:50:18,362] Trial 32 finished with value: 0.8845000267028809 and parameters: {'num_hidden': 1, 'num_units': 194, 'dropout_rate': 0.38051579670677854, 'learning_rate': 0.0011004884764370846, 'activation': 'sigmoid'}. Best is trial 5 with value: 0.8889999985694885.


Epoch 1/10
1500/1500 [==============================] - 5s 3ms/step - loss: 0.8060 - accuracy: 0.7217 - val_loss: 0.4174 - val_accuracy: 0.8528
Epoch 2/10
1500/1500 [==============================] - 5s 3ms/step - loss: 0.4052 - accuracy: 0.8510 - val_loss: 0.3640 - val_accuracy: 0.8654
Epoch 3/10
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3504 - accuracy: 0.8730 - val_loss: 0.3548 - val_accuracy: 0.8700
Epoch 4/10
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3309 - accuracy: 0.8762 - val_loss: 0.3329 - val_accuracy: 0.8794
Epoch 5/10
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3033 - accuracy: 0.8895 - val_loss: 0.3265 - val_accuracy: 0.8798
Epoch 6/10
1500/1500 [==============================] - 5s 3ms/step - loss: 0.2934 - accuracy: 0.8918 - val_loss: 0.3247 - val_accuracy: 0.8827
Epoch 7/10
1500/1500 [==============================] - 5s 3ms/step - loss: 0.2729 - accuracy: 0.8977 - val_loss: 0.3205 - val_accuracy:

[I 2021-05-21 10:51:08,085] Trial 33 finished with value: 0.8914166688919067 and parameters: {'num_hidden': 2, 'num_units': 168, 'dropout_rate': 0.4548896697034186, 'learning_rate': 0.001372144650529276, 'activation': 'sigmoid'}. Best is trial 33 with value: 0.8914166688919067.


Epoch 1/10
 944/1500 [=================>............] - ETA: 1s - loss: 0.9165 - accuracy: 0.6926